## 지하철 승하차 데이터

### 역명과 역사도로명 주소 매칭시키기

In [214]:
import pandas as pd

# Excel 파일 경로
file_path = r"C:\Users\user\Desktop\dataset\전체_도시철도역사정보_20241230.xlsx"

# Excel 파일 읽기
info = pd.read_excel(file_path)

# 원하는 칼럼만 선택
selected_columns = ['역사명', '노선명', '운영기관명', '역사도로명주소']

# 해당 칼럼만 추출
info = info[selected_columns]

# 데이터 확인
print(info.head())



            역사명  노선명   운영기관명                    역사도로명주소
0  청량리(서울시립대입구)  1호선  서울교통공사  서울특별시 동대문구 왕산로 지하205(전농동)
1           제기동  1호선  서울교통공사   서울특별시 동대문구 왕산로 지하93(제기동)
2           신설동  1호선  서울교통공사    서울특별시 동대문구 왕산로 지하1(신설동)
3           동묘앞  1호선  서울교통공사      서울특별시 종로구 종로 359(숭인동)
4           동대문  1호선  서울교통공사    서울특별시 종로구 종로 지하302(창신동)


In [215]:
# '역사도로명주소'에서 불필요한 정보 처리
def clean_address(address):
    try:
        # 괄호 안의 내용 제거하고 '지하' 제거
        cleaned_address = address.split('(')[0].replace('지하', '')
        return cleaned_address
    except Exception as e:
        return address  # 예외 발생 시 원래 주소 반환

# '역사도로명주소' 칼럼에 적용
info['도로명주소'] = info['역사도로명주소'].apply(clean_address)
# '역사도로명주소' 칼럼 삭제
info = info.drop(columns=['노선명','역사도로명주소'])
# '역사명' 기준으로 중복 제거
info = info.drop_duplicates(subset=['역사명'], keep='first')
# 데이터 확인
print(info.head())

            역사명   운영기관명               도로명주소
0  청량리(서울시립대입구)  서울교통공사  서울특별시 동대문구 왕산로 205
1           제기동  서울교통공사   서울특별시 동대문구 왕산로 93
2           신설동  서울교통공사    서울특별시 동대문구 왕산로 1
3           동묘앞  서울교통공사    서울특별시 종로구 종로 359
4           동대문  서울교통공사    서울특별시 종로구 종로 302


In [216]:
# info.to_csv('역별도로명주소.csv', index=False, encoding='utf-8-sig')

In [217]:
import pandas as pd

# CSV 파일 경로
file_path2 = r"C:\Users\user\Desktop\dataset\CARD_SUBWAY_MONTH_202412.csv"

# CSV 파일 읽기
subway = pd.read_csv(file_path2)

# 데이터 확인
print(subway.head())


         사용일자  노선명     역명  승차총승객수    하차총승객수  등록일자
20241201  1호선   종각  16268   14378  20241204   NaN
20241201  1호선   시청  12916   11878  20241204   NaN
20241201  1호선  서울역  40617   39523  20241204   NaN
20241201  2호선   역삼  13249   14046  20241204   NaN
20241201  2호선   선릉  17550   15021  20241204   NaN


In [218]:

import pandas as pd

# CSV 파일 경로
file_path2 = r"C:\Users\user\Desktop\dataset\CARD_SUBWAY_MONTH_202412.csv"

# 두 번째 행부터 불러오기 (첫 번째 행은 데이터로 처리)
subway = pd.read_csv(file_path2, skiprows=1, header=None)

subway.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', '불필요한칼럼']

# '등록일자'와 '불필요한칼럼' 칼럼 삭제
subway = subway.drop(columns=['등록일자', '불필요한칼럼'])

# 데이터 확인
print(subway.head())


       사용일자  노선명   역명  승차총승객수  하차총승객수
0  20241201  1호선   종각   16268   14378
1  20241201  1호선   시청   12916   11878
2  20241201  1호선  서울역   40617   39523
3  20241201  2호선   역삼   13249   14046
4  20241201  2호선   선릉   17550   15021


In [219]:
# 결측치 확인 => 결측치 없음
missing_data = subway.isnull().sum()

# 결측치가 있는 칼럼 확인
print("결측치 개수:")
print(missing_data)

결측치 개수:
사용일자      0
노선명       0
역명        0
승차총승객수    0
하차총승객수    0
dtype: int64


In [220]:
# 사용일자별로 묶기
subway_daily = subway.groupby('사용일자')

# 사용일자별 데이터 개수 확인
daily_counts = subway_daily.size()

# 개수 출력
print(daily_counts)

# 데이터 개수가 모두 동일한지 확인
print("모든 일자의 데이터 개수가 동일한가?", daily_counts.nunique() == 1)


사용일자
20241201    616
20241202    616
20241203    616
20241204    617
20241205    618
20241206    618
20241207    617
20241208    617
20241209    617
20241210    617
20241211    617
20241212    616
20241213    617
20241214    618
20241215    616
20241216    617
20241217    616
20241218    616
20241219    616
20241220    618
20241221    617
20241222    617
20241223    617
20241224    616
20241225    616
20241226    617
20241227    617
20241228    619
20241229    619
20241230    616
20241231    615
dtype: int64
모든 일자의 데이터 개수가 동일한가? False


In [231]:
# 사용일자별, 역명별 승차/하차 총합 계산
subway_daily_station = subway.groupby(['사용일자', '역명'], as_index=False).sum(numeric_only=True)

# 결과 확인
print(subway_daily_station.head())


       사용일자        역명  승차총승객수  하차총승객수
0  20241201  4.19민주묘지    2078    1920
1  20241201        가능    4419    4253
2  20241201      가락시장    9624   10588
3  20241201   가산디지털단지   17927   19012
4  20241201        가양   14685   14257


In [222]:
# 일자별 데이터 저장할 딕셔너리 생성
daily_data = {}

# 사용일자별로 데이터를 나눠 저장
for date, group in subway_daily_station.groupby('사용일자'):
    daily_data[date] = group

# 특정 일자 데이터 확인 (예: 20241201)
print(daily_data[20241201].head())

       사용일자        역명  승차총승객수  하차총승객수
0  20241201  4.19민주묘지    2078    1920
1  20241201        가능    4419    4253
2  20241201      가락시장    9624   10588
3  20241201   가산디지털단지   17927   19012
4  20241201        가양   14685   14257


In [223]:
for date in daily_data.keys():
    # daily_data의 역명과 info의 역사명을 기준으로 병합
    daily_data[date] = daily_data[date].merge(info[['역사명', '도로명주소']], 
                                              left_on='역명', 
                                              right_on='역사명', 
                                              how='left')

    # 불필요한 '역사명' 칼럼 제거
    daily_data[date] = daily_data[date].drop(columns=['역사명'])

# 특정 일자 데이터 확인 (예: 20241201)
print(daily_data[20241201].head())


       사용일자        역명  승차총승객수  하차총승객수               도로명주소
0  20241201  4.19민주묘지    2078    1920   서울특별시 강북구 삼양로 519
1  20241201        가능    4419    4253                 NaN
2  20241201      가락시장    9624   10588  서울특별시 송파구 송파대로 257
3  20241201   가산디지털단지   17927   19012   서울특별시 금천구 벚꽃로 309
4  20241201        가양   14685   14257   서울특별시 강서구 양천로 485


In [224]:
# 20241201일자의 결측치 확인
missing_data = daily_data[20241201].isnull().sum()

# 결과 출력
print(missing_data)
print()

# 20241201일자에서 도로명주소가 결측치인 역명 찾기
missing_address_stations = daily_data[20241201][daily_data[20241201]['도로명주소'].isnull()]

# 결측치인 역명들만 확인
print(missing_address_stations['역명'])


사용일자        0
역명          0
승차총승객수      0
하차총승객수      0
도로명주소     224
dtype: int64

1       가능
6       가좌
7      가천대
8       가평
9       간석
      ... 
510     한티
513     행신
515    호구포
521     화서
523     회기
Name: 역명, Length: 224, dtype: object


In [225]:
import pandas as pd

# 12월 1일자 데이터만 선택
data_20241201 = daily_data[20241201]

# 도로명주소가 결측인 역명만 추출
missing_address_data = data_20241201[data_20241201['도로명주소'].isna()]

# '역' 단어를 역명에 추가하는 함수
def add_station_suffix(name):
    if '역' not in name:  # '역'이 포함되지 않은 경우에만 추가
        return name + '역'
    return name

# 결측인 도로명주소를 가진 역명에 '역' 단어 추가
missing_address_data.loc[:, '역명'] = missing_address_data['역명'].apply(add_station_suffix)

# 수정된 역명을 원본 subway 데이터에 반영
data_20241201.loc[missing_address_data.index, '역명'] = missing_address_data['역명']

# 기존에 병합한 도로명주소 칼럼 제거 (최신 정보로 덮어쓰려면 필요)
data_20241201 = data_20241201.drop(columns=['도로명주소'])

# subway 데이터와 info를 병합하여 도로명주소를 채운다
subway_merged = data_20241201.merge(info[['역사명', '도로명주소']], 
                                    left_on='역명', 
                                    right_on='역사명', 
                                    how='left')

# 불필요한 '역사명' 칼럼 제거
subway_merged = subway_merged.drop(columns=['역사명'])

# 결과 확인
print(subway_merged.head())



       사용일자        역명  승차총승객수  하차총승객수               도로명주소
0  20241201  4.19민주묘지    2078    1920   서울특별시 강북구 삼양로 519
1  20241201       가능역    4419    4253    경기도 의정부시 평화로 633
2  20241201      가락시장    9624   10588  서울특별시 송파구 송파대로 257
3  20241201   가산디지털단지   17927   19012   서울특별시 금천구 벚꽃로 309
4  20241201        가양   14685   14257   서울특별시 강서구 양천로 485


In [226]:

# 도로명주소가 결측인 역명들 추출
missing_address_stations = subway_merged[subway_merged['도로명주소'].isna()]

# 결측인 역명들만 확인
print(missing_address_stations[['역명', '도로명주소']])  


                    역명 도로명주소
45           관악산(서울대)역   NaN
52         교대(법원.검찰청)역   NaN
90   남한산성입구(성남법원.검찰청)역   NaN
104               당고개역   NaN
150       마곡나루(서울식물원)역   NaN
223               삼각지역   NaN
357                 역곡   NaN
413                이수역   NaN
499       하남시청(덕풍·신장)역   NaN


이제 모든 일자에 대해 결측 역명에 "역"을 추가하고 다시 도로명주소를 병합시킴.

In [249]:
# 모든 일자에 대해 반복
for date in daily_data.keys():
    # 해당 일자 데이터 선택
    data = daily_data[date]
    
    # 도로명주소가 결측인 역명만 추출
    missing_address_data = data[data['도로명주소'].isna()]

    # '역' 단어를 역명에 추가하는 함수
    def add_station_suffix(name):
        if '역' not in name:  # '역'이 포함되지 않은 경우에만 추가
            return name + '역'
        return name

    # 결측인 도로명주소를 가진 역명에 '역' 단어 추가
    missing_address_data.loc[:, '역명'] = missing_address_data['역명'].apply(add_station_suffix)

    # 수정된 역명을 원본 데이터에 반영
    data.loc[missing_address_data.index, '역명'] = missing_address_data['역명']

    # 기존에 병합한 도로명주소 칼럼 제거 (최신 정보로 덮어쓰려면 필요)
    data = data.drop(columns=['도로명주소'])

    # subway 데이터와 info를 병합하여 도로명주소를 채운다
    merged_data = data.merge(info[['역사명', '도로명주소']], 
                             left_on='역명', 
                             right_on='역사명', 
                             how='left')

    # 불필요한 '역사명' 칼럼 제거
    merged_data = merged_data.drop(columns=['역사명'])

    # 병합한 데이터를 다시 daily_data에 저장
    daily_data[date] = merged_data

    # 결측치 확인
    missing_data = merged_data.isnull().sum()

    # 도로명주소 결측인 역명들 추출
    missing_data_address = merged_data[merged_data['도로명주소'].isna()]


    # 일자별 데이터 개수 출력
    print(f"Number of records for {date}: {len(data)}")
    # 결측치 출력 => 다 9개가 결측임
    print(f"Missing data for {date}:")
    print(missing_data[['도로명주소']])
    print(missing_data_address[['역명']])  # 결측인 역명만 출력



Number of records for 20241201: 528
Missing data for 20241201:
도로명주소    9
dtype: int64
                    역명
45           관악산(서울대)역
52         교대(법원.검찰청)역
90   남한산성입구(성남법원.검찰청)역
104               당고개역
150       마곡나루(서울식물원)역
223               삼각지역
357                 역곡
413                이수역
499       하남시청(덕풍·신장)역
Number of records for 20241202: 528
Missing data for 20241202:
도로명주소    9
dtype: int64
                    역명
45           관악산(서울대)역
52         교대(법원.검찰청)역
90   남한산성입구(성남법원.검찰청)역
104               당고개역
150       마곡나루(서울식물원)역
223               삼각지역
357                 역곡
413                이수역
499       하남시청(덕풍·신장)역
Number of records for 20241203: 528
Missing data for 20241203:
도로명주소    9
dtype: int64
                    역명
45           관악산(서울대)역
52         교대(법원.검찰청)역
90   남한산성입구(성남법원.검찰청)역
104               당고개역
150       마곡나루(서울식물원)역
223               삼각지역
357                 역곡
413                이수역
499       하남시청(덕풍·신장)역
Number of records for 20241204: 528
Missing data 

In [250]:
import pandas as pd

# 결과를 저장할 리스트 준비
result_list = []

# 모든 일자에 대해 반복
for date in daily_data.keys():
    # 해당 일자 데이터 선택
    data = daily_data[date]
    
    # 일자별 데이터 개수
    record_count = len(data)
    
    # 도로명주소가 결측인 역명만 추출
    missing_address_data = data[data['도로명주소'].isna()]

    # 결측치 확인
    missing_data = data.isnull().sum()

    # 도로명주소 결측인 역명들 추출
    missing_data_address = missing_address_data[['역명']]

    # 결과 리스트에 추가
    result_list.append({
        '사용일자': date,
        '데이터 개수': record_count,
        '결측치 (도로명주소)': missing_data['도로명주소'],
        '결측인 역명': missing_data_address['역명'].tolist()
    })

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_list)

# 결과 출력
print(result_df)


        사용일자  데이터 개수  결측치 (도로명주소)  \
0   20241201     528            9   
1   20241202     528            9   
2   20241203     528            9   
3   20241204     528            9   
4   20241205     528            9   
5   20241206     528            9   
6   20241207     528            9   
7   20241208     528            9   
8   20241209     528            9   
9   20241210     528            9   
10  20241211     528            9   
11  20241212     528            9   
12  20241213     528            9   
13  20241214     528            9   
14  20241215     528            9   
15  20241216     528            9   
16  20241217     528            9   
17  20241218     528            9   
18  20241219     528            9   
19  20241220     528            9   
20  20241221     528            9   
21  20241222     528            9   
22  20241223     528            9   
23  20241224     528            9   
24  20241225     528            9   
25  20241226     528            9   
2

In [245]:
import pandas as pd

# 엑셀 파일로 저장하기 위해 ExcelWriter 사용
with pd.ExcelWriter('24년도12월일자별지하철승하차.xlsx') as writer:
    # 모든 일자에 대해 반복
    for date in daily_data.keys():
        # 해당 일자 데이터 선택
        data = daily_data[date]
        
        # 데이터를 해당 일자에 맞는 시트로 엑셀에 저장
        sheet_name = str(date)  # 시트 이름을 날짜로 설정
        data.to_excel(writer, sheet_name=sheet_name, index=False)  # 시트에 저장
        

In [ ]:
# 기준일자 설정 (예: 20241201)
base_date = 20241201

# 기준일자에서 결측인 역명 추출
base_non_missing_stations = set(daily_data[base_date][daily_data[base_date]['도로명주소'].isna()]['역명'])

# 다른 일자들과 비교
for date in daily_data.keys():
    if date != base_date:  # 기준일자와 비교
        # 해당 일자에서 결측인 역명 추출
        other_missing_stations = set(daily_data[date][daily_data[date]['도로명주소'].isna()]['역명'])
        
        # 기준일자에는 없는 결측인 역명 찾기
        unique_missing_stations_in_other_dates = other_missing_stations - base_non_missing_stations
        
        print(f"Stations missing on {date} but not on {base_date}: {unique_missing_stations_in_other_dates}")

# 다 동일한 역이 결측임.

모든 일자에서 도로명주소가 최종적으로 결측인 역 9개가 동일함.

현재 문제점 
1. 병합 전에 일자별로 데이터 개수가 미세하게 차이 남 => 무슨 역인지 왜 안보이지!!!
2. 최종적으로 도로명주소가 결측인 9개 역명에 도로명주소 기입 필요 => 다른 달들도 하나하나씩 할 수 있을까?

In [243]:
# 12월 1일과 12월 4일 데이터에서 '역명'을 추출하여 set으로 변환
stations_20241201 = set(daily_data[20241201]['역명'])
stations_20241220 = set(daily_data[20241214]['역명'])

# 12월 1일에는 있지만 12월 4일에는 없는 역명
unique_stations_20241201 = stations_20241201 - stations_20241220

# 12월 20일에는 있지만 12월 1일에는 없는 역명
unique_stations_20241220 = stations_20241220 - stations_20241201

# 결과 출력
print(f"Stations in 20241201 but not in 20241204: {unique_stations_20241201}")
print(f"Stations in 20241204 but not in 20241220: {unique_stations_20241220}")


Stations in 20241201 but not in 20241204: set()
Stations in 20241204 but not in 20241220: set()
